In [1]:
## instalacja środowiska wirtualnego
# python3 -m venv venv

## aktywacja środowiska wirtualnego
# source venv/bin/activate

## instalacja bibliotek
# pip install -U pip openai pandas

In [2]:
# import niezbędnych bibliotek
import pandas as pd
from openai import OpenAI

OPENAI_API_KEY = "sk-XXXXXXXXXXXXXXXXXXXXXXX"  # klucz API

client = OpenAI(api_key=OPENAI_API_KEY)
model = "gpt-3.5-turbo"                                                 # model językowy (gpt-4-turbo)

In [3]:
# dane wejściowe znajdują się w pliku parts.csv

# wczytanie danych z pliku csv
df_ = pd.read_csv("parts.csv", sep=";", header=None, names=["part_number", "catalog_name", "Null", "part_numebr", "description"])
df = df_[["part_number", "catalog_name", "description"]].copy()

# wybranie kolumn na których będziemy pracować. W tym przypadku są to: catalog_name i description.
# format stringów w kolumnach jest różny, więc trzeba je przekonwertować na string.
df["description"] = df["description"].astype(str)
df["catalog_name"] = df["catalog_name"].astype(str)

# wyświetlenie pierwszych 5 wierszy
df.head()

,part_number,catalog_name,description
0,0 204 769 754,"SWERW0 HAM. NISSAN QASHQAI 1,6 13-",nan
1,0 204 817 839,POMPA HAM. BMW 3 /F30/ 18-,nan
2,0 204 840 167,"SERWO HAM. FORD TRANSIT 2,0 19-",nan
3,0 986 134 178,ZACISK HAM. CITROEN DS5 11-,nan
4,0 986 280 471,CZUJNIK OBROT?W WA?U. PEUGOT 306 1.9D 88-93,nan


In [7]:
# utworzenie funkcji get_description, która zwraca opis części na podstawie nazwy katalogowej

def get_description(catalog_name):
  system_prompt = '''You are a helpful assistant. You are also a car mechanic with extended knowledge of cars parts.
  Your task is genrate a description of the part based on it catalog name as close as possible to the catalog name.
  Alwasys answer in Polish. 
  No preambles, just the description.'''
  
  user_prompt = f'''For the catalog name {catalog_name}, I need a description of the part.
  
  Use the following examples marked with xml tags:
  
  <example>
  - SWERW0 HAM. NISSAN QASHQAI 1,6 13- -> Serwo hamulca
  - POMPA HAM.BMW 3 /F30/ 18- -> Pompa hamulcowa
  - POMPA HAM. FORD FOCUS 1,6 04- -> Pompa hamulcowa
  - SERWO HAM.FORD TRANSIT 2,0 19- -> Serwo hamulca
  </example>
  
  Use exact catalog name in the description. Do not use any other words.
  '''

  response = client.chat.completions.create(
    model=model,
    temperature=0.1,
    max_tokens=30,
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": user_prompt},
    ]
  )
  return str(response.choices[0].message.content)

In [11]:
# wywołanie funkcji get_description dla każdego wiersza w kolumnie catalog_name
df.loc[:, "description"] = df["catalog_name"].apply(lambda x: get_description(x))

# wyświetlenie 5 losowych wierszy z kolumnami part_number, catalog_name i description
df.sample(5)

,part_number,catalog_name,description
2,0 204 840 167,"SERWO HAM. FORD TRANSIT 2,0 19-",Serwo hamulca
6,0 986 280 744,CZUJNIK WA?U KORB. RENAULT KANGOO 1.5D 0 2-,Czujnik wału korbowego.
3,0 986 134 178,ZACISK HAM. CITROEN DS5 11-,Zacisk hamulcowy.
1,0 204 817 839,POMPA HAM. BMW 3 /F30/ 18-,Pompa hamulcowa
0,0 204 769 754,"SWERW0 HAM. NISSAN QASHQAI 1,6 13-",Serwo hamulca


In [10]:
# zapisanie wyników do pliku csv parts_with_descriptions.csv

df.to_csv("parts_with_descriptions.csv", sep=";", index=False)